In [1]:
print("hlelloo")

hlelloo


In [ ]:
from torch import nn
import torch
import monai
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
class Sobel(nn.Module):
    def __init__(self):
        super().__init__()
        self.filter = nn.Conv3d(in_channels=1, out_channels=3, kernel_size=3, stride=1, padding=1, padding_mode="reflect", bias=False)

        # defining three 3x3x3 sobel operators for x, y, z direction
        Gx = torch.tensor([
            [
                [1.0, 2.0, 1.0],
                [2.0, 4.0, 2.0],
                [1.0, 2.0, 1.0]
            ],
            [
                [0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0]
            ],
            [
                [-1.0, -2.0, -1.0],
                [-2.0, -4.0, -2.0],
                [-1.0, -2.0, -1.0]
            ]
        ])
        Gy = torch.tensor([
            [
                [1.0, 0.0, -1.0],
                [2.0, 0.0, -2.0],
                [1.0, 0.0, -1.0]
            ],
            [
                [2.0, 0.0, -2.0],
                [4.0, 0.0, -4.0],
                [2.0, 0.0, -2.0]
            ],
            [
                [1.0, 0.0, -1.0],
                [2.0, 0.0, -2.0],
                [1.0, 0.0, -1.0]
            ]
        ])
        Gz = torch.tensor([
            [
                [1.0, 2.0, 1.0],
                [0.0, 0.0, 0.0],
                [-1.0, -2.0, -1.0]
            ],
            [
                [2.0, 4.0, 2.0],
                [0.0, 0.0, 0.0],
                [-2.0, -4.0, -2.0]
            ],
            [
                [1.0, 2.0, 1.0],
                [0.0, 0.0, 0.0],
                [-1.0, -2.0, -1.0]
            ]
        ])

        # combine sobel kernels and hand this parameter over to nn.Conv3d() function
        G = torch.cat([Gx.unsqueeze(0), Gy.unsqueeze(0), Gz.unsqueeze(0)], 0)
        G = G.unsqueeze(1)
        self.filter.weight = nn.Parameter(G, requires_grad=False)

    def forward(self, img):
        # calculate pixel values
        x = self.filter(img)
        x = torch.mul(x, x)
        x = torch.sum(x, dim=0, keepdim=True)
        x = torch.sqrt(x)
        return x

In [ ]:
img_path = r".\Crete_no7_20um_1821x1671x1455_16bit_trans_376.nii.gz"
load_image = monai.transforms.LoadImage(dtype=np.float32, ensure_channel_first=True)
img = load_image(img_path)
print(img.shape)

DataParallel(
  (module): SRCNN3D(
    (conv1): Conv3d(1, 64, kernel_size=(9, 9, 9), stride=(1, 1, 1), padding=(4, 4, 4))
    (conv2): Conv3d(64, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
    (conv3): Conv3d(32, 1, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
# extract slices from original data
arr_orig = np.array(img[0])
middle_slice_orig_x = arr_orig[int(len(arr_orig) / 2), :, :]
middle_slice_orig_y = arr_orig[:, int(len(arr_orig) / 2), :]
middle_slice_orig_z = arr_orig[:, :, int(len(arr_orig) / 2)]

# create sobel gradient volume
img_sobel = Sobel().forward(img)
arr_sobel = np.array(img_sobel[0])
middle_slice_sobel_x = arr_sobel[int(len(arr_sobel) / 2), :, :]
middle_slice_sobel_y = arr_sobel[:, int(len(arr_sobel) / 2), :]
middle_slice_sobel_z = arr_sobel[:, :, int(len(arr_sobel) / 2)]

# visualize
fig, axes = plt.subplots(3, 2, figsize=(10, 10))
axes[0, 0].imshow(middle_slice_orig_x, cmap="gray")
axes[0, 0].set_title("Original Slices (x, y, z view)")
axes[0, 1].imshow(middle_slice_sobel_x, cmap="gray")
axes[0, 1].set_title("Sobel Slices (x, y, z view)")
axes[1, 0].imshow(middle_slice_orig_y, cmap="gray")
axes[1, 1].imshow(middle_slice_sobel_y, cmap="gray")
axes[2, 0].imshow(middle_slice_orig_z, cmap="gray")
axes[2, 1].imshow(middle_slice_sobel_z, cmap="gray")